In [92]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

#paraphraseTokenizer = AutoTokenizer.from_pretrained(
    #"bert-base-cased-finetuned-mrpc",cache_dir ='../model/')
#paraphraseModel = AutoModelForSequenceClassification.from_pretrained(
    #"bert-base-cased-finetuned-mrpc", cache_dir ='../model/',)
    
paraphraseTokenizer = AutoTokenizer.from_pretrained("textattack/roberta-base-MRPC")  
paraphraseModel = AutoModelForSequenceClassification.from_pretrained("textattack/roberta-base-MRPC")

from sentence_transformers import SentenceTransformer, util
sentenceTransformer = SentenceTransformer("roberta-large-nli-stsb-mean-tokens")

classes = ["not paraphrase", "is paraphrase"]



Some weights of the model checkpoint at textattack/roberta-base-MRPC were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [93]:
def word_similarity(s1, s2):
    num_sim = 0
    seq1 = s1.split(" ")
    for w in seq1:
        if w in s2:
            num_sim += 1
    return num_sim / len(seq1)

def inference_mrpc(seq1s, seq2s):
    for i in range(len(seq1s)):
        paraphrase = paraphraseTokenizer.encode_plus(
            seq1s[i], seq2s[i], return_tensors="pt")
        logits = paraphraseModel(**paraphrase)[0]
        paraphrase_results = torch.softmax(logits, dim=1).tolist()[0]
        print(f"{classes[1]}: {round(paraphrase_results[1] * 100)}%")

def inference_sts(sentences1, sentences2):
    embeddings1 = sentenceTransformer.encode(sentences1, convert_to_tensor=True)
    embeddings2 = sentenceTransformer.encode(sentences2, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)
    for i in range(len(sentences1)):
        print("Similarity Score: {:.4f}".format(cosine_scores[i][i] * word_similarity(
            sentences1[i], sentences2[i])))

In [96]:
seq0 = "I caused him to submit his resignation"
seq1 = "I caused the submission of his resignation"
seq2 = "I caused him to submit the resignation"
seq3 = "I caused the submission of his resignation"
seq4 = "I caused the submission of the resignation"
seq5 = "I caused the submission of his resignation"
seq6 = "John ate an apple"
seq7 = "an apple was eaten by john"

sentences1 = [seq0, seq2, seq4, seq6]
sentences2 = [seq1, seq3, seq5, seq7]

inference_mrpc(sentences1, sentences2)
inference_sts(sentences1, sentences2)

is paraphrase: 100%
is paraphrase: 100%
is paraphrase: 100%
is paraphrase: 99%
Similarity Score: 0.5415
Similarity Score: 0.5464
Similarity Score: 0.9770
Similarity Score: 0.6805
